<a href="https://colab.research.google.com/github/jppasso/Computer-Vision-University-Assignment-CAPG002/blob/main/Vegetables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

# Import Kaggle Dataset

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"joaopassos","key":"503dc7e31a72caa4a7b8fe1f3b41074f"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              233KB  2021-08-15 09:48:42          10678  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           6529  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2459  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2651  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [ ]:
! kaggle datasets download -d raghavrpotdar/fresh-and-stale-images-of-fruits-and-vegetables

fresh-and-stale-images-of-fruits-and-vegetables.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! mkdir vegetables

mkdir: cannot create directory ‘vegetables’: File exists


In [ ]:
! unzip fresh-and-stale-images-of-fruits-and-vegetables.zip -d vegetables

Archive:  fresh-and-stale-images-of-fruits-and-vegetables.zip
replace vegetables/ImageLabels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace vegetables/fresh_apple/Screen Shot 2018-06-08 at 4.59.36 PM.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Libraries Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pathlib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

print(tf.__version__)

2.5.0


# Data Set Up

Set Data Directory

In [ ]:
data_dir = pathlib.Path('/content/vegetables')
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)

10144


Data Exploration

In [ ]:
fresh_apples = list(data_dir.glob('fresh_apple/*'))
PIL.Image.open(str(fresh_apples[0]))

In [ ]:
PIL.Image.open(str(fresh_apples[1]))

In [ ]:
fresh_bananas = list(data_dir.glob('fresh_banana/*'))
PIL.Image.open(str(fresh_bananas[0]))

In [ ]:
PIL.Image.open(str(fresh_bananas[1]))

Load Parameters

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

val_split = 0.2
seed_ref = 123

Dataset Split

In [ ]:
# Standard Keras Tutorial Pre-processing
# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=val_split,
  subset="training",
  seed=seed_ref,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Validation Dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=val_split,
  subset="validation",
  seed=seed_ref,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 14682 files belonging to 12 classes.
Using 11746 files for training.
Found 14682 files belonging to 12 classes.
Using 2936 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['fresh_apple', 'fresh_banana', 'fresh_bitter_gourd', 'fresh_capsicum', 'fresh_orange', 'fresh_tomato', 'stale_apple', 'stale_banana', 'stale_bitter_gourd', 'stale_capsicum', 'stale_orange', 'stale_tomato']


Visualize Data

In [ ]:
# First 9 Images in the Training Dataset
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Configure For Performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Standardize Data


In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

In [ ]:
# Standard MobileNet V2 Pre-processing
dataset = dataset.map(lambda x, y: (preprocessing_layer(x), y))

In [ ]:
for images, labels in normalized_train_ds.take(1):
  for i in range(9):
    print(np.min(images[i].numpy().astype("uint8")), np.max(images[i].numpy().astype("uint8")))

Trained Model Application

# Model

Create Model

In [ ]:
num_classes = 12

In [ ]:
# Standard Keras Tutorial Model
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
# MobileNet V2 Model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=num_classes,
    classifier_activation="softmax",
)

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1, input_shape=(img_height, img_width, 3)),        
    base_model,
])

Compile Model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model Summary

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 12)                2273356   
Total params: 2,273,356
Trainable params: 2,239,244
Non-trainable params: 34,112
_________________________________________________________________


Model Training

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


368/368 [==============================] - 2074s 6s/step - loss: 0.8594 - accuracy: 0.6973 - val_loss: 2.9477 - val_accuracy: 0.1141
Epoch 2/10
368/368 [==============================] - 1942s 5s/step - loss: 0.4077 - accuracy: 0.8543 - val_loss: 4.1848 - val_accuracy: 0.1611
Epoch 3/10
368/368 [==============================] - 1943s 5s/step - loss: 0.2762 - accuracy: 0.9030 - val_loss: 4.6686 - val_accuracy: 0.1141
Epoch 4/10
368/368 [==============================] - 1943s 5s/step - loss: 0.2317 - accuracy: 0.9163 - val_loss: 5.3426 - val_accuracy: 0.1611
Epoch 5/10
368/368 [==============================] - 1942s 5s/step - loss: 0.1901 - accuracy: 0.9325 - val_loss: 5.9639 - val_accuracy: 0.1090
Epoch 6/10
261/368 [====================>.........] - ETA: 9:06 - loss: 0.1395 - accuracy: 0.9506

KeyboardInterrupt: ignored

Training Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Predict on new data

In [ ]:
stale_banana_path = '/content/vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 8.47.28 PM.png'

img = keras.preprocessing.image.load_img(
    stale_banana_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)